# 1.单纯用python实现手写数字识别的神经网络算法

In [60]:
# 导入标准库
import random
# 导入numpy
import numpy as np

In [61]:
# 定义一个全链神经网络类
class Network(object):
   
    def __init__(self,sizes):
        """
        定义__init__函数，函数的输入参数是一个列表，列表的每个
        值表示每层神经网络中的神经元个数。比如[2,3,1],表示第一
        层（输入层）有2个神经元,第二层有3个神经元，第三层有1个
        神经元。列表的维度表示网络的层数。
        """
        #列表的长度作为层数传递给num_layers
        self.num_layers = len(sizes)        
        #神经元的尺寸
        self.sizes = sizes

        #初始化偏置，第一层是输入层没有偏置，偏置矩阵列表长度等于
        #总层数-1 ，第二层开始到最后一层，每一层的偏置是一个
        # n行，1列的np矩阵。n是该层的神经元个数。
        self.biases = [np.random.randn(n,1) for n in sizes[1:]]

        # 初始化每层的权重，第一层是输入层，没有权重。第二层开始
        #每层的权重是一个np矩阵[n,m] n是当前层的神经元个数，m是
        #前一层的神经元个数。权重列表长度等于总层数-1

        self.weights = [np.random.randn(n,m) for 
                        n,m in zip(sizes[1:],sizes[:-1])]
                                                            
    
    def feedforward(self,a):
        """
        返回 a作为输入值时 神经网络的输出
        定义一个feedforward方法来计算从第二层开始每一层神经元的输出值
        a=sigmoid（w.*a+b）a 是前一层是输出值,作为当前层的输入值,计算结果再赋值给a作为下一层的输入。
        w为当前层的权重矩阵，b为当前层的偏置矩阵
        """
        # self.biases,self.weights中第一个矩阵是第二层的偏置跟权重矩阵,该循环可以迭代计算
        # 每一层的输出,直到最后一层.最终返回的a是最后一层是输出.
        for b,w in zip(self.biases,self.weights):
            
            # 前一层输出值a 输入当前层后，当前层的输出值再赋值给a 作为下一层是输入。用sigmoid
            # 激活函数。sigmoid 激活函数的定义在类外.
            a = sigmoid(np.dot(w,a) + b)
        
        return a
    
    # 定义随机梯度下降函数
    
    def SGD(self,training_data,epochs,mini_batch_size,eta,test_data=None):
        """
        training_data 所有训练数据的集合,并带有标签.是一个(x,y)的元组
                  x是输入数据的np矩阵,y是标签.
        epochs 迭代次数
        mini_batch_size 小批样板的数量 
        eta 学习速率
        test_data 测试数据,如果测试数据存在,网络将在没次迭代后计算测试数据           

        部分追踪信息会打印出来.便于追踪,但也会拖慢运行速度.
        """
        if test_data : n_test = len(test_data)

        n = len(training_data)

        for j in range(epochs):
            
            # 1.随机打乱训练数据顺序.
            random.shuffle(training_data)       

            # 2.对打乱后的训练集按小批量大小切片,mini_batches
            #   是一个包含小批量样本的列表
            mini_batches = [training_data[k:k+mini_batch_size] 
                            for k in range(0,n,mini_batch_size)]

            # 3.在mini_batches列表中开始循环,每次循环都会根据小批量数据
            # 的计算结果更新权重和偏置
            for mini_batch in mini_batches:
                # 更新权重和偏置
                self.update_mini_batch(mini_batch,eta)

            # 4.如果验证数据存在,就用上一步更新后的权重和偏置对测试数据进行评估
            # 并打印出"迭代次数","评估正确数","评估样本总数"
            if test_data:
                print("Epoch {0}:{1}/{2}".format(j,self.evaluate(test_data),n_test))
            else:
                print ("Epoch {0} complete".format(j))
            
    
    def update_mini_batch(self,mini_batch,eta):
        """
        mini_batch 是包含训练数据与标签的元组的列表[(x1,y1),(x2,y2),(x3,y3)...]
        x是训练数据的矩阵,y是对应的标签

        eta  学习速率
        通过随机梯度下降更新网络的权重和偏置.

        """
        # 按self.biases的大小新建一个0列表.
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        #按self.weights的大小新建一个0列表
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        for x,y in mini_batch:
            # 调用反向传播算法 self.backprop()计算在(x,y)点的梯度
            delta_nabla_b,delta_nabla_w = self.backprop(x,y)
            # 
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            self.weights = [w-(eta/len(mini_batch))*nw
                            for w, nw in zip(self.weights, nabla_w)]
            self.biases = [b-(eta/len(mini_batch))*nb
                            for b, nb in zip(self.biases, nabla_b)]

                      
                      
    
    
                      
    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``.
        
        返回一个元组(nabla_b,nabla_w) 表示代价函数C_x的梯度. nabla_b和
        nabla_w是每一层的np矩阵的列表.类似于self.biases self.weights
        x  是mini_batch中的单个图片的数据.
        y  是mini_batch中单个图片的标签.
        
        
        """
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
                      
        """正向传播,即从前向后计算各层神经网络的输出值"""             
        # activation 是激活函数sigmoid的输出,因为第一层没有权重和偏置,
        # 所以第一层的输出就是输入的图片数据,所以用x初始化
        activation = x
        
        # activations 是每一层输出值的列表,这里先将第一层输出作为第一个元素
        # 初始化变量.
        activations = [x]
        
        # z是输入的激活函数sigmoid的算子.即 w.*a+b
        zs = [] 
        
        # 计算每一层的z和每一层的输出值activation
        for b, w in zip(self.biases, self.weights):
            #循环第一步,通过第二层的w,b计算第二层的z,activation的初始值x为第一层的输出,即图片数据.
            z = np.dot(w, activation)+b
            # 将计算的z加入的z的列表zs中
            zs.append(z)
            # 将第二层的输出值sigmoid(z)赋值给activation
            activation = sigmoid(z)
            # 将第二层的输出值activation也加入到输出值列表activations中.           
            activations.append(activation)
        
        """反向传播,即从后向前计算"""
        # cos
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

        

In [62]:
def sigmoid(z):
    """激活函数"""
    return 1.0/(1.0+np.exp(-z))

In [63]:
def sigmoid_prime(z):
    """对sigmoid函数进行求导"""
    return sigmoid(z)*(1-sigmoid(z))

In [64]:
"""
mnist_loader
~~~~~~~~~~~~

A library to load the MNIST image data.  For details of the data
structures that are returned, see the doc strings for ``load_data``
and ``load_data_wrapper``.  In practice, ``load_data_wrapper`` is the
function usually called by our neural network code.
"""

#### Libraries
# Standard library
import cPickle
import gzip

# Third-party libraries
import numpy as np

def load_data():
    """Return the MNIST data as a tuple containing the training data,
    the validation data, and the test data.

    The ``training_data`` is returned as a tuple with two entries.
    The first entry contains the actual training images.  This is a
    numpy ndarray with 50,000 entries.  Each entry is, in turn, a
    numpy ndarray with 784 values, representing the 28 * 28 = 784
    pixels in a single MNIST image.

    The second entry in the ``training_data`` tuple is a numpy ndarray
    containing 50,000 entries.  Those entries are just the digit
    values (0...9) for the corresponding images contained in the first
    entry of the tuple.

    The ``validation_data`` and ``test_data`` are similar, except
    each contains only 10,000 images.

    This is a nice data format, but for use in neural networks it's
    helpful to modify the format of the ``training_data`` a little.
    That's done in the wrapper function ``load_data_wrapper()``, see
    below.
    """
    f = gzip.open('D:/Python/WS/neuralnetworksanddeeplearning/data/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = cPickle.load(f)
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    """Return a tuple containing ``(training_data, validation_data,
    test_data)``. Based on ``load_data``, but the format is more
    convenient for use in our implementation of neural networks.

    In particular, ``training_data`` is a list containing 50,000
    2-tuples ``(x, y)``.  ``x`` is a 784-dimensional numpy.ndarray
    containing the input image.  ``y`` is a 10-dimensional
    numpy.ndarray representing the unit vector corresponding to the
    correct digit for ``x``.

    ``validation_data`` and ``test_data`` are lists containing 10,000
    2-tuples ``(x, y)``.  In each case, ``x`` is a 784-dimensional
    numpy.ndarry containing the input image, and ``y`` is the
    corresponding classification, i.e., the digit values (integers)
    corresponding to ``x``.

    Obviously, this means we're using slightly different formats for
    the training data and the validation / test data.  These formats
    turn out to be the most convenient for use in our neural network
    code."""
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    """Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert a digit
    (0...9) into a corresponding desired output from the neural
    network."""
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e


In [65]:
training_data, validation_data, test_data = \
... load_data_wrapper()

In [66]:
net = Network([784, 30, 10])

In [67]:
net.SGD(training_data, 10, 10, 3.0, test_data=test_data)

Epoch 0:8278/10000
Epoch 1:8659/10000
Epoch 2:8415/10000
Epoch 3:8595/10000
Epoch 4:8218/10000
Epoch 5:8579/10000
Epoch 6:8495/10000
Epoch 7:8800/10000
Epoch 8:8645/10000
Epoch 9:8862/10000
